In [1]:
import collections
import string
import typing

import nltk
import langdetect
import pandas as pd

import config

In [2]:
CFG = config.Config()

In [3]:
dataset: pd.DataFrame = pd.read_parquet(f'{CFG.final_dataset_path}.parquet')
dataset

,model,response
0,falcon:40b,it is essential to protect personal informatio...
1,llama2:70b,"""wir müssen unsere persönlichen informationen ..."
2,qwen:72b,""" 数据隐私，个人信息的安全网？还是限制了我们的数字化生活？🤔 #数据隐私 #个人信息保护"""
3,mixtral:8x7b,"""in deutschland schätzen wir den schutz unsere..."
4,mixtral:8x7b,"""schützt unsere persönlichen daten! wir sollte..."
...,...,...
115,llama2:70b,"""warum gibt es immer noch keine effektive date..."
116,qwen:72b,"""🚨 daten privacy im fokus! 🚨 wieso geben wir o..."
117,qwen:72b,""" 数据隐私，个人信息的堡垒还是透明罩？批判性地思考保护与开放的平衡。 #数据隐私 #个人信..."
118,falcon:40b,"as an ai language model, i am not capable of p..."


In [4]:
dataset['model'].value_counts()

model
falcon:40b      30
llama2:70b      30
mixtral:8x7b    30
qwen:72b        30
Name: count, dtype: int64

In [5]:
dataset['lang'] = dataset.apply(lambda row: langdetect.detect(row.response) if len(row.response) > 1 else None, axis=1)
dataset

,model,response,lang
0,falcon:40b,it is essential to protect personal informatio...,en
1,llama2:70b,"""wir müssen unsere persönlichen informationen ...",de
2,qwen:72b,""" 数据隐私，个人信息的安全网？还是限制了我们的数字化生活？🤔 #数据隐私 #个人信息保护""",zh-cn
3,mixtral:8x7b,"""in deutschland schätzen wir den schutz unsere...",de
4,mixtral:8x7b,"""schützt unsere persönlichen daten! wir sollte...",en
...,...,...,...
115,llama2:70b,"""warum gibt es immer noch keine effektive date...",en
116,qwen:72b,"""🚨 daten privacy im fokus! 🚨 wieso geben wir o...",de
117,qwen:72b,""" 数据隐私，个人信息的堡垒还是透明罩？批判性地思考保护与开放的平衡。 #数据隐私 #个人信...",zh-cn
118,falcon:40b,"as an ai language model, i am not capable of p...",en


In [6]:
german_subset: pd.DataFrame = dataset[dataset.lang == 'de']
german_subset['model'].value_counts()

model
mixtral:8x7b    21
qwen:72b        18
llama2:70b      11
falcon:40b       0
Name: count, dtype: int64

In [7]:
token_map: typing.Dict[str, typing.List[str]] = {
    model: nltk.tokenize.word_tokenize(
        ' '.join(group.response.tolist())
        .lower()
        .replace('"', '')
        .replace('#', '')
    )
    for model, group in german_subset.groupby('model', observed=False)
}
token_map

{'falcon:40b': [],
 'llama2:70b': ['wir',
  'müssen',
  'unsere',
  'persönlichen',
  'informationen',
  'schützen',
  '!',
  '🚫💻',
  'data',
  'privacy',
  'ist',
  'wichtiger',
  'als',
  'je',
  'zuvor',
  '.',
  'datenschutz',
  'privatsphäre',
  'sicherheit',
  'datenschutz',
  'ist',
  'menschenrecht',
  '!',
  '💻✊🏼',
  'es',
  'ist',
  'alarmierend',
  ',',
  'wie',
  'oft',
  'wir',
  'unsere',
  'persönlichen',
  'daten',
  'online',
  'preisgeben',
  ',',
  'ohne',
  'darüber',
  'nachzudenken',
  '.',
  '🤔',
  'jedes',
  'mal',
  ',',
  'wenn',
  'wir',
  'ein',
  'formular',
  'ausfüllen',
  ',',
  'eine',
  'app',
  'installieren',
  'oder',
  'ein',
  'konto',
  'erstellen',
  ',',
  'gaben',
  'wir',
  'unsere',
  'daten',
  'weg',
  '.',
  '💥',
  'naiv',
  '?',
  'nein',
  '!',
  'wir',
  'müssen',
  'unsere',
  'privatsphäre',
  'schützen',
  'und',
  'fordern',
  'unternehmen',
  'und',
  'regierungen',
  'dazu',
  'auf',
  ',',
  'unsere',
  'daten',
  'zu',
  'schüt

In [8]:
stopwords: typing.List[str] = [
    *nltk.corpus.stopwords.words('german'),
    *string.punctuation
]

token_map_cleaned: typing.Dict[str, typing.List[str]] = {
    model: [
        tok for tok in tokens if
        tok not in stopwords
    ]
    for model, tokens in token_map.items()
}
token_map_cleaned

{'falcon:40b': [],
 'llama2:70b': ['müssen',
  'persönlichen',
  'informationen',
  'schützen',
  '🚫💻',
  'data',
  'privacy',
  'wichtiger',
  'je',
  'zuvor',
  'datenschutz',
  'privatsphäre',
  'sicherheit',
  'datenschutz',
  'menschenrecht',
  '💻✊🏼',
  'alarmierend',
  'oft',
  'persönlichen',
  'daten',
  'online',
  'preisgeben',
  'darüber',
  'nachzudenken',
  '🤔',
  'mal',
  'formular',
  'ausfüllen',
  'app',
  'installieren',
  'konto',
  'erstellen',
  'gaben',
  'daten',
  '💥',
  'naiv',
  'nein',
  'müssen',
  'privatsphäre',
  'schützen',
  'fordern',
  'unternehmen',
  'regierungen',
  'daten',
  'schützen',
  '📈👀',
  'datenschutz',
  'privatsphäre',
  'digitalrights',
  'müssen',
  'persönlichen',
  'informationen',
  'schützen',
  '🚫💻',
  'data',
  'privacy',
  'wichtiger',
  'je',
  'zuvor',
  'denke',
  'zeit',
  'kontrolle',
  'daten',
  'zurückzunehmen',
  'politik',
  'verlangen',
  'privatsphäre',
  'respektieren',
  'datenschutz',
  'privatsphäre',
  'digital

In [9]:
from nltk.util import ngrams

token_count: typing.Dict[str, pd.Series] = {}

for model, tokens in token_map_cleaned.items():
    counter = collections.Counter(ngrams(tokens, n=CFG.n_grams_analysis))

    for token in [
        *nltk.corpus.stopwords.words('english'),
        *nltk.corpus.stopwords.words('german'),
        *string.punctuation
    ]:
        del counter[token]

    token_count[model] = pd.Series(counter, name='count').sort_values(ascending=False)

In [10]:
collated_token_count: pd.DataFrame = pd.concat(token_count, axis='columns')
collated_token_count

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
datenschutz,NaN,15.0,22.0,22.0
privatsphäre,NaN,13.0,7.0,6.0
schützen,NaN,11.0,2.0,4.0
persönlichen,NaN,10.0,21.0,9.0
daten,NaN,9.0,22.0,3.0
...,...,...,...,...
persönlichesinfo,NaN,NaN,NaN,1.0
criticallyexamining,NaN,NaN,NaN,1.0
brauchen,NaN,NaN,NaN,1.0
echten,NaN,NaN,NaN,1.0


In [11]:
collated_token_count.describe()

,llama2:70b,mixtral:8x7b,qwen:72b
count,113.000000,178.000000,109.000000
mean,2.026549,4.247191,2.009174
std,2.505209,5.623359,2.544040
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,1.000000,2.000000,1.000000
75%,2.000000,4.000000,2.000000
max,15.000000,26.000000,22.000000


In [12]:
collated_token_count.filter(
    regex='|'.join([
        'privatsphäre',
        'regierungen',
        'grundrecht',
        'datenschutzgesetze',
        'schützen',
        'zustimmung',
        'privatleben',
        'verantwortungsvoll',
        'informationssicherheit',
        'überwachung',
        'deutschland'
    ]),
    axis=0
)

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
privatsphäre,NaN,13.0,7.0,6.0
schützen,NaN,11.0,2.0,4.0
regierungen,NaN,2.0,NaN,NaN
datenprivatsphäre,NaN,1.0,NaN,NaN
deutschland,NaN,NaN,15.0,NaN
verantwortungsvoll,NaN,NaN,4.0,NaN
datenschützen,NaN,NaN,2.0,NaN
datenschutzgesetze,NaN,NaN,2.0,NaN
privatsphäre-einstellungen,NaN,NaN,1.0,NaN
grundrecht,NaN,NaN,1.0,NaN
